In [46]:
%matplotlib inline
import pyro
import torch
import numpy as np
import matplotlib.pyplot as plt
import pyro.optim as optim
import pyro.distributions as dist
from torch.distributions import constraints
from tqdm import tqdm_notebook as tqdm  
import seaborn as sns
from matplotlib import animation, rc
from IPython.display import HTML
import torch.nn as nn
from functools import partial
import pandas as pd
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoDelta
from pyro.infer import EmpiricalMarginal, SVI, Trace_ELBO, TracePredictive
import pandas as pd

from pyro import poutine

In [47]:
import torch
import torch.nn as nn
from torch.autograd import Variable

from numpy import pi
import numpy as np


In [48]:
pyro.set_rng_seed(1)
pyro.enable_validation(True)
torch.set_default_dtype(torch.float64)

In [49]:
def load(file_name, delete_columns=None, y_column=-1):
    if delete_columns is None:
        delete_columns = []
    df = pd.read_csv(file_name, header=None)
    matrix = df.to_numpy()
    result_matrix = np.delete(matrix, delete_columns, axis=1)
    result_matrix = np.delete(result_matrix, y_column, axis=1)
    result_matrix = np.hstack((result_matrix, matrix[:, y_column][:, np.newaxis]))
    return result_matrix


def split_data(raw_data_matrix, test_set_percent, shuffle):
    if shuffle:
        process_data = np.random.permutation(raw_data_matrix)
    else:
        process_data = raw_data_matrix
    split_index = int(process_data.shape[0] * test_set_percent)
    return process_data[:split_index, :], process_data[split_index:, :]


def extract_attributes_label(data_matrix) -> dict:
    class_vector, class_labels = pd.factorize(data_matrix[:, -1])
    data_matrix = np.vstack((data_matrix[:, :-1].T, class_vector)).T.astype(float)
    return {"x_y": data_matrix, "x": data_matrix[:, :-1], "y": data_matrix[:, -1], "y_labels": np.vstack(
        (np.arange(class_labels.size), class_labels)).T, "y_values": np.arange(class_labels.size)}


def positive(predict_vector, class_vector):
    return np.count_nonzero((predict_vector - class_vector) == 0) / predict_vector.shape[0]

def calc_metric(y_true: np.ndarray, y_pred: np.ndarray, average: str = "macro"):
    return Metric(
        accuracy=sm.accuracy_score(y_true, y_pred),
        precision=sm.precision_score(y_true, y_pred, average=average),
        recall=sm.recall_score(y_true, y_pred, average=average),
        f1=sm.f1_score(y_true, y_pred, average=average)
    )

In [50]:
data_matrix = (load("data/iris.csv"), "iris")
extracted_data = extract_attributes_label(data_matrix[0])

x_data = torch.from_numpy(extracted_data["x"])
y_data = torch.from_numpy(extracted_data["y"])
classes_nbr = len(y_data.unique())
features_nbr = len(x_data[1,:])
count = len(y_data)



In [51]:
class NBModel(nn.Module):
    def __init__(self,classes,features):
        super(self.__class__, self).__init__()
        self.classes = classes
        self.features = features
        
        self.register_parameter(
            "means",
            nn.Parameter(torch.zeros(self.classes, self.features))
        )
        self.register_parameter(
            "variances",
            nn.Parameter(torch.ones(self.classes, self.features))
        )
        self.register_parameter(
            "classes_priors",
            nn.Parameter(torch.ones(self.classes) * (1/self.classes))
        )

    def forward(self, x):
        x = x[:,np.newaxis,:]
        return (torch.sum(- 0.5 * torch.log(2. * pi * self.variances)
                - (x - self.means)**2 / torch.abs(self.variances) / 2, dim=-1)
                + torch.log(self.classes_priors))
    
nbModel = NBModel(classes_nbr,features_nbr)

In [52]:
def nb_model2(x_data, y_data):
    
    means = pyro.distributions.Normal(loc=torch.zeros_like(nbModel.means), scale=torch.ones_like(nbModel.means)).to_event(2)
    variances = pyro.distributions.InverseGamma(concentration=torch.ones_like(nbModel.variances),rate=torch.ones_like(nbModel.variances)).to_event(2)
    classes_priors = pyro.distributions.Dirichlet(concentration=torch.ones_like(nbModel.classes_priors))
    
    priors = {'means': means, 'variances': variances, "classes_priors": classes_priors}
    lifted_module = pyro.random_module("module", nbModel, priors)
    lifted_nb_model = lifted_module()
    
    #with pyro.plate("map_cl", len(x_data)):
        
    
    with pyro.plate("map", len(x_data)):
        class_prob = lifted_nb_model(x_data)
        pyro.sample("obs_cl",pyro.distributions.Categorical(probs=lifted_nb_model.classes_priors),obs=y_data)
        pyro.sample("obs",pyro.distributions.Categorical(logits=class_prob),obs=y_data)
        return class_prob

In [100]:
def nb_guide2(x_data, y_data):
    means_m = pyro.param("means_m",torch.zeros_like(nbModel.means))
    means_s = pyro.param("means_s",torch.ones_like(nbModel.means), constraint=constraints.positive)
    variances_c = pyro.param("variances_a", torch.ones_like(nbModel.variances), constraint=constraints.positive)
    variances_r = pyro.param("variances_b", torch.ones_like(nbModel.variances), constraint=constraints.positive)
    classes_priors_c = pyro.param("classes_priors_c", torch.ones_like(nbModel.classes_priors)/len(nbModel.classes_priors), constraint=constraints.positive)
    
    means = pyro.distributions.Normal(loc=means_m, scale=means_s).to_event(2)
    variances = pyro.distributions.InverseGamma(concentration=variances_c,rate=variances_r).to_event(2)
    classes_priors = pyro.distributions.Dirichlet(concentration=classes_priors_c)
    
    priors = {'means': means, 'variances': variances, "classes_priors": classes_priors}
    lifted_module = pyro.random_module("module", nbModel, priors)
    return lifted_module()


In [104]:
def train(params):
    step_data = { "loss":[] }
    for param in  params:
        step_data[param]=[]
    pyro.clear_param_store()
    num_iterations=5_000
    model = nb_model2
    guide = nb_guide2
    optim = pyro.optim.Adam({"lr": 0.01})
    svi = pyro.infer.SVI(model, guide, optim, loss=pyro.infer.Trace_ELBO(), num_samples=count)
    for j in tqdm(range(num_iterations)):
        loss = svi.step(x_data, y_data.squeeze(-1))
        step_data["loss"].append(loss)
        for param in params:
            step_data[param].append(pyro.param(param).clone())
        if j%1000 == 0:
            print(f"{j}:{loss}")
    return (model, svi, step_data)

In [105]:
probabilistic_model, svi, data = train(())

0:3559.1796263065194
1000:275.5432863164556
2000:267.9606270005756
3000:237.71254793660043
4000:254.24414906504435


In [1]:
plt.plot(data["loss"])
plt.title("evidence lower bound (ELBO)")
plt.xlabel("step")
plt.yscale("log")
plt.ylabel("loss")
plt.show()
plt.plot([data.detach().numpy() for data in data["auto_loc"]])
plt.title("auto_loc")
plt.xlabel("step")
plt.ylabel("loc")
plt.show()
plt.plot([data.detach().numpy() for data in data["auto_scale"]])
plt.title("auto_scale")
plt.xlabel("step")
plt.ylabel("scale");

NameError: name 'plt' is not defined

In [107]:
def wrapped_model(x_data, y_data):
    model_result=probabilistic_model(x_data, y_data)
    pyro.sample("prediction", pyro.distributions.Delta(model_result))
    
posterior = svi.run(x_data, y_data)

trace_pred = TracePredictive(wrapped_model,
                             posterior,
                             num_samples=1)

In [108]:
post_pred = trace_pred.run(x_data, None)

In [109]:
em = EmpiricalMarginal(post_pred,["prediction"])
prob = em._get_samples_and_weights()[0].detach().cpu().numpy()
prob = np.squeeze(prob)

In [110]:
result = np.argmax(prob,axis=1)

In [111]:
positive(result,y_data.numpy())

0.9666666666666667

In [112]:
result

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [113]:
y_data.numpy()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])

In [114]:
torch.tensor([[1,2,3],[1,2,3]])

tensor([[1, 2, 3],
        [1, 2, 3]])

In [115]:
em2=EmpiricalMarginal(post_pred,["module$$$classes_priors"])
prob2 = np.squeeze(em2._get_samples_and_weights()[0].detach().cpu().numpy())

In [116]:
prob2

array([0.31805685, 0.36316461, 0.31877854])

In [ ]:
    import numpy
    actual = numpy.array(actual)
    predicted = numpy.array(predicted)

    # calculate the confusion matrix; labels is numpy array of classification labels
    cm = numpy.zeros((len(labels), len(labels)))
    for a, p in zip(actual, predicted):
        cm[a][p] += 1

    # also get the accuracy easily with numpy
    accuracy = (actual == predicted).sum() / float(len(actual))

In [863]:
sklearn.metrics.confusion_matrix(y_data.numpy(),result)

tensor([1., 0., 0.])

tensor([0.1049, 0.1010, 0.1299, 0.3575, 0.3067])

In [8]:
pyro.distributions.Dirichlet(concentration=torch.tensor([0.1,-1.,1.])).sample()

ValueError: The parameter concentration has invalid values